In [1]:
import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'
import os
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

In [2]:
o3d.__version__

'0.16.1+61276a05'

In [2]:
(4.7**2-1.09**2)**(1/2)

4.571859577896067

In [3]:
(1.53**2-1.09**2)**(1/2)

1.0736852425175638

In [4]:
(1.09**2-4.7**2)**(1/2)

(2.799456599110794e-16+4.571859577896067j)

In [3]:
intrinsic = np.array([[790.3578186 ,   0.        , 364.2981863 ],
                      [  0.        , 790.35787582, 472.93979645],
                      [  0.        ,   0.        ,   1.        ]])
extrisic = np.array([[ 0.00302267,  0.7469503 , -0.6648731 , -0.07202526],
                     [ 0.9999116 , -0.01087168, -0.00766794,  0.02143548],
                     [-0.01295585, -0.66479117, -0.74691695, -0.15440787],
                     [ 0.        ,  0.        ,  0.        ,  1.        ]],)

In [4]:
472.93979645*2

945.8795929

In [6]:
o3d.visualization.Visualizer()

Visualizer with name Open3D

In [3]:
import open3d as o3d
import open3d.visualization.rendering as rendering

Open3D was not compiled with BUILD_GUI, but script is importing open3d.visualization.rendering


In [4]:
o3d.visualization.rendering

<module 'open3d.visualization.rendering' from '/miniconda/envs/py310/lib/python3.10/site-packages/open3d/visualization/rendering/__init__.py'>

In [9]:
 open3d.__DEVICE_API__

NameError: name 'open3d' is not defined

In [6]:
import open3d as o3d
import open3d.visualization.rendering as rendering

model = o3d.io.read_triangle_mesh("/workspace/advtex_init_align/dataset/extracted/scene_04/mesh.ply")

render = rendering.OffscreenRenderer(960,720)
# render = o3d.visualization.Visualizer()
render.scene.add_model("model",model)
render.scene.scene.set_sun_light([0.707, 0.0, -0.707], [1.0, 1.0, 1.0], 75000)
render.scene.scene.enable_sun_light(True)
# render.setup_camera(40.0, [0, 0.5 0], [1, 0.75, 1.75], [0, 1, 0])
render.setup_camera(intrinsic, extrisic, 960, 720)
img = render.renter_to_image()
o3d.io.write_image("test.png", img, 9)

AttributeError: module 'open3d.visualization.rendering' has no attribute 'OffscreenRenderer'

In [ ]:
import os
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt


def custom_draw_geometry_with_camera_trajectory(pcd, camera_trajectory_path,
                                                render_option_path,
                                                output_path):
    custom_draw_geometry_with_camera_trajectory.index = -1
    custom_draw_geometry_with_camera_trajectory.trajectory =\
        o3d.io.read_pinhole_camera_trajectory(camera_trajectory_path)
    custom_draw_geometry_with_camera_trajectory.vis = o3d.visualization.Visualizer()
    image_path = os.path.join(output_path, 'image')
    if not os.path.exists(image_path):
        os.makedirs(image_path)
    depth_path = os.path.join(output_path, 'depth')
    if not os.path.exists(depth_path):
        os.makedirs(depth_path)

    print("Saving color images in " + image_path)
    print("Saving depth images in " + depth_path)

    def move_forward(vis):
        # This function is called within the o3d.visualization.Visualizer::run() loop
        # The run loop calls the function, then re-render
        # So the sequence in this function is to:
        # 1. Capture frame
        # 2. index++, check ending criteria
        # 3. Set camera
        # 4. (Re-render)
        ctr = vis.get_view_control()
        glb = custom_draw_geometry_with_camera_trajectory
        if glb.index >= 0:
            print("Capture image {:05d}".format(glb.index))
            # Capture and save image using Open3D.
            vis.capture_depth_image(
                os.path.join(depth_path, "{:05d}.png".format(glb.index)), False)
            vis.capture_screen_image(
                os.path.join(image_path, "{:05d}.png".format(glb.index)), False)

            # Example to save image using matplotlib.
            '''
            depth = vis.capture_depth_float_buffer()
            image = vis.capture_screen_float_buffer()
            plt.imsave(os.path.join(depth_path, "{:05d}.png".format(glb.index)),
                       np.asarray(depth),
                       dpi=1)
            plt.imsave(os.path.join(image_path, "{:05d}.png".format(glb.index)),
                       np.asarray(image),
                       dpi=1)
            '''

        glb.index = glb.index + 1
        if glb.index < len(glb.trajectory.parameters):
            ctr.convert_from_pinhole_camera_parameters(
                glb.trajectory.parameters[glb.index])
        else:
            custom_draw_geometry_with_camera_trajectory.vis.destroy_window()

        # Return false as we don't need to call UpdateGeometry()
        return False

    vis = custom_draw_geometry_with_camera_trajectory.vis
    vis.create_window()
    vis.add_geometry(pcd)
    vis.get_render_option().load_from_json(render_option_path)
    vis.register_animation_callback(move_forward)
    vis.run()


if __name__ == "__main__":
    if not o3d._build_config['ENABLE_HEADLESS_RENDERING']:
        print("Headless rendering is not enabled. "
              "Please rebuild Open3D with ENABLE_HEADLESS_RENDERING=ON")
        exit(1)

    sample_data = o3d.data.DemoCustomVisualization()
    pcd = o3d.io.read_point_cloud(sample_data.point_cloud_path)
    print("Customized visualization playing a camera trajectory. "
          "Press ctrl+z to terminate.")
    custom_draw_geometry_with_camera_trajectory(
        pcd, sample_data.camera_trajectory_path, sample_data.render_option_path,
        'HeadlessRenderingOutput')